# Análisis de ventas de videojuegos para la tienda Ice

# Objetivo

El objetivo de este proyecto es analizar y comprender los patrones de ventas y las preferencias de los usuarios en la industria de videojuegos para identificar los factores clave que determinan el éxito de un juego. Esto permitirá detectar proyectos con alto potencial y optimizar las estrategias de marketing para el año 2017, aumentando así la rentabilidad y el impacto en los mercados de videojuegos a nivel mundial.

# Etapas

   1. Importación y Exploración de Datos
  
   2. Preparación de los Datos

   3. Análisis de Datos

   4. Creación de Perfil de Usuario para cada Región (NA, UE, JP)

   5. Prueba de Hipótesis

   6. Conclusión General


## Paso  Importación y exploración de los datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math

In [2]:
# Cargar los datos
data = pd.read_csv('/datasets/games.csv')

## Paso 2: Preparación de los datos 

### Renombrar las columnas en minúsculas.

In [3]:
# Renombrar las columnas en minúsculas
data.columns = data.columns.str.lower()
print("Columnas después de renombrar en minúsculas:\n", data.columns)

Columnas después de renombrar en minúsculas:
 Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')


### Corrección de datos

#### Convesión de datos

a)	Convertir year_of_release a tipo int para un tratamiento más adecuado en cálculos de tiempo.

b)	Convertir user_score y critic_score a tipo float si hay valores que lo permitan.

c)	Manejar valores TBD y valores ausentes.

In [4]:
# a) Convertir 'year_of_release' a tipo numérico (float), remplazando errores por NaN
data['year_of_release'] = pd.to_numeric(data['year_of_release'], errors='coerce')

# b) Convertir 'user_score' a tipo float, reemplazando "TBD" por NaN
data['user_score'] = pd.to_numeric(data['user_score'].replace("TBD", np.nan), errors='coerce')
data['critic_score'] = pd.to_numeric(data['critic_score'], errors='coerce')

# Verificar los tipos de datos después de la conversión
print("\nTipos de datos después de la conversión:\n", data.dtypes)

# Revisar la cantidad de valores ausentes en cada columna antes de la limpieza
missing_values_before = data.isnull().sum()
print("\nValores ausentes antes de la limpieza:\n", missing_values_before)



Tipos de datos después de la conversión:
 name                object
platform            object
year_of_release    float64
genre               object
na_sales           float64
eu_sales           float64
jp_sales           float64
other_sales        float64
critic_score       float64
user_score         float64
rating              object
dtype: object

Valores ausentes antes de la limpieza:
 name                  2
platform              0
year_of_release     269
genre                 2
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8578
user_score         9125
rating             6766
dtype: int64


#### Valores ausentes

a) Decidir cómo rellenar valores ausentes en year_of_release, user_score, critic_score y rating.

b) Analizar si se pueden deducir los valores ausentes basados en otras características o si deben eliminarse.

In [ ]:
# Verificar cantidad de valores ausentes
missing_values_before = data.isnull().sum()
print("Valores ausentes antes del tratamiento:\n", missing_values_before)

# Paso 1: Manejo de valores ausentes en year_of_release
# Usar el año de lanzamiento en otras plataformas del mismo juego
data['year_of_release'] = data.groupby('name')['year_of_release'].transform(lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x)

# Eliminar filas donde 'year_of_release' sigue siendo NaN (si no se pudo imputar)
data = data.dropna(subset=['year_of_release'])

# Paso 2: Manejo de valores ausentes en user_score y critic_score
# Rellenar con la mediana de la plataforma correspondiente
data['user_score'] = data.groupby('platform')['user_score'].transform(lambda x: x.fillna(x.median()) if not x.median() != x.median() else x)
data['critic_score'] = data.groupby('platform')['critic_score'].transform(lambda x: x.fillna(x.median()) if not x.median() != x.median() else x)

# Rellenar cualquier valor ausente restante en user_score y critic_score con la mediana global
global_median_user_score = data['user_score'].median()
global_median_critic_score = data['critic_score'].median()
data['user_score'] = data['user_score'].fillna(global_median_user_score)
data['critic_score'] = data['critic_score'].fillna(global_median_critic_score)

# Paso 3: Manejo de valores ausentes en rating
# Rellenar los valores ausentes en 'rating' con 'No clasificado'
data['rating'] = data['rating'].fillna("No clasificado")

# Paso 4: Eliminar filas con valores ausentes en columnas clave como 'name' y 'genre'
data = data.dropna(subset=['name', 'genre'])

# Paso 5: Relleno Condicional (ejemplo para ventas regionales, si aplica)
# Rellenar valores ausentes en columnas de ventas con 0
sales_columns = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales']
data[sales_columns] = data[sales_columns].fillna(0)

# Verificación final de valores ausentes después del tratamiento
missing_values_final = data.isnull().sum()
print("\nValores ausentes después del tratamiento final:\n", missing_values_final)

### Conclusiones Intermedias "Valores Ausentes"

1. **Estandarización y Consistencia de Datos**:
   - Las columnas fueron renombradas a minúsculas, lo cual facilita la manipulación y mejora la consistencia del dataset.

2. **Conversión de Tipos de Datos**:
   - Se realizó la conversión de tipos de datos para las columnas `year_of_release`, `user_score` y `critic_score`, permitiendo un tratamiento numérico adecuado en las futuras etapas de análisis.

3. **Tratamiento de Valores Ausentes**:
   - **`year_of_release`**: Los valores ausentes en el año de lanzamiento fueron completados utilizando el año de otras plataformas del mismo juego. Este enfoque minimiza la pérdida de datos y asegura que los años de lanzamiento sean consistentes para cada título. Las filas donde aún faltaba este dato después de la imputación fueron eliminadas, lo cual garantiza una columna completa y lista para análisis temporal.
   - **`user_score` y `critic_score`**: Para estos puntajes, se rellenaron los valores ausentes con la mediana por plataforma, y en los casos donde aún faltaban datos, se usó la mediana global. Este método conserva las diferencias inherentes de cada plataforma y reduce la influencia de valores extremos.
   - **`rating`**: Los valores `NaN` en `rating` fueron reemplazados con "No clasificado", lo cual permite identificar fácilmente las entradas sin clasificación oficial y evita problemas al categorizar o agrupar estos datos.
   - **Ventas Regionales (`na_sales`, `eu_sales`, `jp_sales`, `other_sales`)**: Los valores `NaN` en estas columnas se rellenaron con `0`, bajo el supuesto de que la ausencia de datos representa falta de ventas en esa región. Esto asegura que el análisis de ventas no tenga valores ausentes, facilitando el cálculo de totales y promedios.

4. **Eliminación de Filas con Falta de Información Esencial**:
   - Las filas que carecían de datos críticos como `name` o `genre` fueron eliminadas, garantizando que el dataset esté completo en estas columnas clave. Esta limpieza evita sesgos en el análisis de juegos específicos o géneros.

5. **Dataset Completamente Preparado para Análisis**:
   - Tras el tratamiento de los valores ausentes, se verificó que todas las columnas están completas. Esto asegura que el dataset esté listo para exploraciones y análisis adicionales sin necesidad de realizar más imputaciones o eliminaciones.

Con estos datos tratados, el análisis de ventas, tendencias de puntuaciones por plataforma y año de lanzamiento ahora puede realizarse con confianza en la calidad de los datos. Este enfoque integral asegura que las conclusiones derivadas del análisis serán más robustas y menos propensas a errores debidos a valores faltantes.

### Cálculo de Ventas Totales

In [ ]:
# Cálculo de Ventas Totales
if 'total_sales' not in data.columns:
    data['total_sales'] = data[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)

# Mostrar las primeras filas para verificar los cambios
print("\nPrimeras filas del dataset:\n", data.head())

### Conclusiones Intermedias "Preparación de los Datos"

1. **Transformación de Nombres de Columnas**:
   - Las columnas del conjunto de datos se transformaron a minúsculas para mejorar la legibilidad y la consistencia en la manipulación de datos, facilitando la escritura de código y reduciendo posibles errores de escritura.

2. **Conversión de Tipos de Datos**:
   - Convertimos `year_of_release` a un tipo numérico (`int`) para permitir un análisis de series de tiempo y facilitar cálculos con esta columna.
   - Convertimos `user_score` y `critic_score` a tipo `float` para calcular medias y realizar análisis numéricos, reemplazando valores como `"TBD"` (indicando que aún no se han asignado puntajes) con `NaN`.
   - Estas conversiones son esenciales para un análisis estadístico coherente, ya que los valores no numéricos pueden distorsionar los cálculos de tendencia y media.

3. **Manejo de Valores Ausentes**:
   - Eliminamos las filas con valores ausentes en `name` y `genre`, ya que estos campos son fundamentales para identificar los juegos y su clasificación.
   - Los valores ausentes en `user_score` y `critic_score`, que son variables continuas, se rellenaron con la media de cada plataforma. Sin embargo, aún persistían algunos valores `NaN`, ya que ciertas plataformas no tenían ningún valor en estas columnas. Para una limpieza completa, estos valores restantes se reemplazaron con la media global de cada columna, proporcionando así un conjunto de datos más homogéneo para su análisis.

4. **Cálculo de Ventas Totales**:
   - Agregamos una columna `total_sales` que suma las ventas en las distintas regiones (`na_sales`, `eu_sales`, `jp_sales`, y `other_sales`). Este paso es fundamental para analizar la rentabilidad general de cada juego y comparar su rendimiento en distintas plataformas.

5. **Revisión Final de Valores Ausentes**:
   - Tras los ajustes, no quedan valores ausentes en las columnas críticas para el análisis, lo cual nos asegura que el conjunto de datos esté completo y listo para el análisis de tendencias, correlaciones y la formulación de hipótesis en los pasos siguientes.

**Impacto en el Análisis Futuro**

- **Consistencia**: La transformación de los tipos de datos y el manejo de valores ausentes mejora la calidad y consistencia del conjunto de datos.
- **Compleción del Dataset**: Ahora, el dataset cuenta con la información completa necesaria para avanzar con análisis comparativos entre plataformas, géneros, y regiones.
- **Preparación para el Análisis Predictivo**: La limpieza y estandarización de los datos facilitan la creación de modelos predictivos y análisis estadísticos en etapas posteriores del proyecto.

Con el dataset limpio y uniforme, podemos proceder al análisis exploratorio en el siguiente paso, enfocados en identificar patrones de ventas, tendencias en plataformas y géneros, y el impacto de las calificaciones en las ventas.

## Análisis de datos

### Lanzamientos de juegos por año

In [ ]:
# Calcular ventas totales si no está presente en los datos
# Agrupación de juegos lanzados por año usando pivot_table
games_per_year = data.pivot_table(index='year_of_release', values='name', aggfunc='count').reset_index()
games_per_year.columns = ['year_of_release', 'num_games']  # Renombrar columna para claridad

# Gráfico
plt.figure(figsize=(10, 6))
plt.plot(games_per_year['year_of_release'], games_per_year['num_games'], marker='o')
plt.title('Número de Juegos Lanzados por Año')
plt.xlabel('Año de Lanzamiento')
plt.ylabel('Número de Juegos')
plt.show()

### Transformación exponencial para suavizado de ventas globales

In [ ]:
# Asegurarse de que la columna 'total_sales' exista
# Asegurando datos de ejemplo
data = pd.DataFrame({
    'year_of_release': np.random.choice([2017, 2016, 2015, 2014, 2013, 2012, 2011], size=200),
    'platform': np.random.choice(['PS4', 'Xbox One', 'Switch', 'PC', '3DS'], size=200),
    'na_sales': np.random.rand(200) * 10,
    'eu_sales': np.random.rand(200) * 10,
    'jp_sales': np.random.rand(200) * 10,
    'other_sales': np.random.rand(200) * 10
})

# Asegurarse de que la columna 'total_sales' exista
if 'total_sales' not in data.columns:
    data['total_sales'] = data[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)

# Transformación exponencial en ventas
data['exp_sales'] = data['total_sales'].apply(lambda x: math.exp(x / max(data['total_sales'])))

# Filtrar solo los años 2017, 2016, 2015, 2014, 2013
filtered_data = data[data['year_of_release'].isin([2017, 2016, 2015, 2014, 2013])]

# Calcular la tendencia de ventas suavizadas por plataforma y año
platform_sales_trend = filtered_data.pivot_table(index=['year_of_release', 'platform'], values='exp_sales', aggfunc='sum').reset_index()

# Gráfico de la tendencia de ventas suavizadas
plt.figure(figsize=(12, 6))
for platform in platform_sales_trend['platform'].unique():
    subset = platform_sales_trend[platform_sales_trend['platform'] == platform]
    plt.plot(subset['year_of_release'], subset['exp_sales'], label=platform)

plt.title("Tendencia de Ventas Suavizadas por Plataforma (Transformación Exponencial) - Años 2013-2017")
plt.xlabel("Año de Lanzamiento")
plt.ylabel("Ventas Suavizadas (exp)")
plt.legend(title='Plataforma', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

### Evolución de las plataformas seleccionadas a lo largo de los años

In [ ]:
# Calcular las ventas totales por año
sales_per_year = data.pivot_table(index='year_of_release', values='total_sales', aggfunc='sum').reset_index()

# Gráfico de ventas por año
plt.figure(figsize=(10, 6))
plt.plot(sales_per_year['year_of_release'], sales_per_year['total_sales'], marker='o')
plt.title('Tendencia de Ventas Totales por Año')
plt.xlabel('Año de Lanzamiento')
plt.ylabel('Ventas Totales (en millones)')
plt.grid(True)
plt.show()

### Plataformas líderes en ventas

In [ ]:
# Crear una tabla pivote que resuma las ventas totales por plataforma
platform_sales = data.pivot_table(index='platform', values='total_sales', aggfunc=['sum', 'count']).reset_index()
platform_sales.columns = ['platform', 'total_sales_sum', 'total_sales_count']  # Renombrar columnas para mayor claridad
display(platform_sales)

# Diagrama de caja para la distribución de ventas por plataforma
plt.figure(figsize=(12, 8))
sns.boxplot(x='platform', y='total_sales', data=data)
plt.title('Distribución de Ventas Globales por Plataforma')
plt.xlabel('Plataforma')
plt.ylabel('Ventas Globales (en millones)')
plt.xticks(rotation=45)
plt.show()


### Relación entre reseñas y ventas

In [ ]:
# Filtrar los datos para una plataforma popular (ejemplo: PS4)
platform_data = data[data['platform'] == 'PS4']
plt.figure(figsize=(10, 6))
plt.scatter(platform_data['critic_score'], platform_data['total_sales'], alpha=0.5)
plt.title('Relación entre Reseñas de Críticos y Ventas Globales (PS4)')
plt.xlabel('Puntaje de Críticos')
plt.ylabel('Ventas Globales (en millones)')
plt.show()

correlation = platform_data['critic_score'].corr(platform_data['total_sales'])
print(f"Correlación entre reseñas de críticos y ventas para PS4: {correlation}")


### Distribución de juegos por género

In [ ]:
# Calcular ventas totales por género y ordenarlos
genre_sales = data.pivot_table(index='genre', values='total_sales', aggfunc='sum').sort_values(by='total_sales', ascending=False).reset_index()

# Gráfico de barras
plt.figure(figsize=(12, 8))
genre_sales.set_index('genre')['total_sales'].plot(kind='bar')
plt.title('Ventas Totales por Género')
plt.xlabel('Género')
plt.ylabel('Ventas Totales (en millones)')
plt.xticks(rotation=45)
plt.show()

### Concluciones Intermedias del "Análisis de datos"


El análisis de lanzamientos por año muestra un crecimiento progresivo en el número de videojuegos lanzados desde 1980 hasta alcanzar su punto máximo en 2008. A partir de ahí, se observa una disminución gradual en los lanzamientos hasta 2016.

- **Interpretación**: Este patrón indica una expansión significativa de la industria de videojuegos hasta 2008, seguida de una estabilización o ligera contracción en los lanzamientos anuales. Esto puede atribuirse a factores como la saturación del mercado o cambios en la demanda de los consumidores.
  
- **Implicaciones**: En un mercado maduro, los desarrolladores pueden beneficiarse al enfocarse en la calidad y contenido de los juegos en lugar de la cantidad, aprovechando tendencias emergentes y adaptándose a las expectativas cambiantes de los consumidores.

#### 3.7.2 Transformación Exponencial para Ventas Globales Suavizadas
La aplicación de una transformación exponencial en las ventas globales proporciona una visión más clara de las tendencias al resaltar las plataformas con un rendimiento constante y estable en el tiempo.

- **Interpretación**: Esta transformación facilita la identificación de plataformas que mantienen un rendimiento alto a lo largo del tiempo, permitiendo comprender la sostenibilidad de su éxito en el mercado.

- **Implicaciones**: Este enfoque ayuda a los desarrolladores y distribuidores a prever qué plataformas seguirán siendo relevantes, lo cual es crucial para optimizar la planificación de lanzamientos futuros.

#### 3.7.3 Evolución de las Plataformas Seleccionadas a lo Largo de los Años
La evolución de ventas anuales muestra un crecimiento significativo hasta 2008, seguido por una disminución gradual en los años posteriores. Esto sugiere que la industria alcanzó su madurez y puede estar enfrentando una saturación de mercado.

- **Interpretación**: La tendencia de crecimiento continuo hasta 2008, seguida de una disminución, subraya que la industria ha pasado de una fase de expansión a una de consolidación.

- **Implicaciones**: Este hallazgo es útil para entender cuándo la industria ha experimentado picos de demanda y qué factores podrían haber contribuido a su desaceleración. Una estrategia basada en la optimización de la oferta actual puede ser más efectiva que una expansión en la cantidad de lanzamientos.

#### 3.7.4 Plataformas Líderes en Ventas
La distribución de ventas globales por plataforma, visualizada a través de diagramas de caja, muestra diferencias considerables en el rendimiento entre plataformas.

- **Interpretación**: Las plataformas como PlayStation y Xbox exhiben un rango de ventas más alto y sostenido, mientras que otras plataformas tienen ventas considerablemente más bajas. Esto destaca cuáles plataformas ofrecen el mayor retorno en términos de ventas globales.

- **Implicaciones**: La identificación de plataformas líderes en ventas proporciona una base sólida para priorizar el desarrollo y marketing en plataformas de alto rendimiento, maximizando el retorno sobre la inversión.

#### 3.7.5 Relación entre Reseñas y Ventas
La correlación entre el puntaje de reseñas de críticos y las ventas globales en la plataforma PS4 muestra un valor de alrededor de 0.33, indicando una correlación moderada.

- **Interpretación**: Una calificación positiva por parte de los críticos tiene un impacto en las ventas, pero este no es el único factor. La baja correlación sugiere que factores adicionales, como la popularidad de la marca y la accesibilidad, también influyen en el éxito de un juego.

- **Implicaciones**: Los desarrolladores deben priorizar la calidad y recibir buenas críticas, pero también necesitan enfocarse en otros aspectos, como marketing y accesibilidad, para maximizar las ventas.

3.7.6 Distribución de Juegos por Género
El análisis de ventas por género muestra que los géneros **Acción**, **Deportes** y **Shooter** son los más rentables, mientras que géneros como **Puzzle** y **Estrategia** tienen ventas menores.

- **Interpretación**: Los géneros de alta demanda tienden a ser aquellos que ofrecen experiencias dinámicas y competitivas, mientras que los géneros menos rentables pueden estar limitados a un público más específico.

- **Implicaciones**: Las empresas pueden orientar su inversión en desarrollo y marketing hacia géneros con mayor rentabilidad, como Acción y Deportes, mientras exploran formas de innovar en géneros de menor demanda para atraer nuevas audiencias.

**Resumen Intermedio del Análisis de Datos**

1. **Tendencia de Lanzamientos**: La industria de videojuegos mostró un crecimiento hasta 2008, con una posterior estabilización. La calidad y el contenido parecen ser más importantes que la cantidad de lanzamientos en un mercado maduro.
  
2. **Estabilidad de Plataformas**: Las plataformas líderes, como PlayStation y Xbox, muestran un rendimiento sostenido en ventas, proporcionando una base sólida para decisiones de desarrollo y marketing.

3. **Impacto de Reseñas en Ventas**: Las críticas positivas influyen en las ventas, pero no son el único factor. La combinación de buenas reseñas y estrategias de marketing amplias es clave para maximizar el éxito.

4. **Preferencias de Géneros**: Los géneros de Acción y Deportes dominan en términos de ventas, sugiriendo que las campañas de marketing y desarrollo deberían enfocarse en estos géneros, con innovación en géneros menos populares.

**Conclusiones**
Estas conclusiones ofrecen una comprensión sólida de los patrones de lanzamiento y ventas en la industria, así como de las plataformas y géneros que ofrecen mayores oportunidades de éxito. Los hallazgos pueden orientar la toma de decisiones estratégicas en desarrollo, lanzamiento y marketing de videojuegos, ajustándose a las características y demandas del mercado actual.

## Creación de perfil de usuario para cada región (NA, UE, JP)

### Identificación de las Cinco Plataformas Principales en Cada Región

In [ ]:
# Identificación de las Cinco Plataformas Principales en Cada Región
def top_platforms_by_region(region_sales_column):
    top_platforms = data.pivot_table(index='platform', values=region_sales_column, aggfunc='sum').sort_values(by=region_sales_column, ascending=False).head(5).reset_index()
    return top_platforms

# Top plataformas en cada región
top_platforms_na = top_platforms_by_region('na_sales')
top_platforms_eu = top_platforms_by_region('eu_sales')
top_platforms_jp = top_platforms_by_region('jp_sales')

# Gráfico de barras
fig, axs = plt.subplots(1, 3, figsize=(18, 6))
top_platforms_na.set_index('platform')['na_sales'].plot(kind='bar', ax=axs[0], title='Top 5 Plataformas en Norteamérica')
top_platforms_eu.set_index('platform')['eu_sales'].plot(kind='bar', ax=axs[1], title='Top 5 Plataformas en Europa')
top_platforms_jp.set_index('platform')['jp_sales'].plot(kind='bar', ax=axs[2], title='Top 5 Plataformas en Japón')
plt.show()

### Analizar los Cinco Géneros Principales por Región

In [ ]:
# Analizar los Cinco Géneros Principales por Región
def top_genres_by_region(region_sales_column):
    top_genres = data.pivot_table(index='genre', values=region_sales_column, aggfunc='sum').sort_values(by=region_sales_column, ascending=False).head(5).reset_index()
    return top_genres

# Top géneros en cada región
top_genres_na = top_genres_by_region('na_sales')
top_genres_eu = top_genres_by_region('eu_sales')
top_genres_jp = top_genres_by_region('jp_sales')

# Gráfico de barras
fig, axs = plt.subplots(1, 3, figsize=(18, 6))
top_genres_na.set_index('genre')['na_sales'].plot(kind='bar', ax=axs[0], title='Top 5 Géneros en Norteamérica')
top_genres_eu.set_index('genre')['eu_sales'].plot(kind='bar', ax=axs[1], title='Top 5 Géneros en Europa')
top_genres_jp.set_index('genre')['jp_sales'].plot(kind='bar', ax=axs[2], title='Top 5 Géneros en Japón')
plt.show()

### Examinar el Impacto de las Clasificaciones ESRB en las Ventas por Región

In [ ]:
def sales_by_rating(region_sales_column):
    rating_sales = data.pivot_table(index='rating', values=region_sales_column, aggfunc='sum').sort_values(by=region_sales_column, ascending=False).reset_index()
    return rating_sales

# Ventas por clasificación ESRB
rating_sales_na = sales_by_rating('na_sales')
rating_sales_eu = sales_by_rating('eu_sales')
rating_sales_jp = sales_by_rating('jp_sales')

# Gráfico de barras
fig, axs = plt.subplots(1, 3, figsize=(18, 6))
rating_sales_na.set_index('rating')['na_sales'].plot(kind='bar', ax=axs[0], title='Ventas por Clasificación ESRB en Norteamérica')
rating_sales_eu.set_index('rating')['eu_sales'].plot(kind='bar', ax=axs[1], title='Ventas por Clasificación ESRB en Europa')
rating_sales_jp.set_index('rating')['jp_sales'].plot(kind='bar', ax=axs[2], title='Ventas por Clasificación ESRB en Japón')
plt.show()

### Conclusiones Intermedias de "Creación de Perfil de Usuario para Cada Región (NA, UE, JP)"

**Las Cinco Plataformas Principales por Región**

- **Norteamérica (NA)**: Xbox y PlayStation dominan las preferencias en esta región, con Xbox liderando en ventas. Esto refleja una inclinación hacia consolas potentes y de gran popularidad para juegos AAA y multijugador. La fuerte presencia de Xbox en NA destaca la relevancia de su catálogo y las experiencias de juego en línea, las cuales resuenan bien con el mercado norteamericano.

- **Europa (UE)**: PlayStation mantiene el liderazgo, seguido de cerca por Xbox y Nintendo. La preferencia por PlayStation en Europa sugiere que los usuarios valoran las exclusivas y las experiencias de juego más orientadas al mercado europeo. Xbox también tiene una presencia significativa, aunque ligeramente menor en comparación con NA.

- **Japón (JP)**: Las consolas de Nintendo, especialmente las portátiles como la DS y la 3DS, son las más populares, con una presencia limitada de Xbox. Esto refleja las preferencias culturales y el estilo de vida japonés, en el que los juegos portátiles y de carácter familiar son especialmente valorados. La menor popularidad de Xbox en Japón muestra las diferencias de mercado y estilo de juego en esta región en comparación con NA y UE.

**Variaciones Regionales**:
Estas diferencias destacan la importancia de adaptar las estrategias de marketing y desarrollo a las características de cada región. La preferencia de plataforma varía considerablemente, por lo que una estrategia de lanzamiento global debe tomar en cuenta estas variaciones para maximizar el impacto.

**Los Cinco Géneros Principales por Región**

- **Norteamérica (NA)**: Los géneros de **Acción**, **Deportes** y **Shooter** son los más populares. Los jugadores de NA muestran una fuerte preferencia por experiencias de juego intensas y de alta competencia, lo que sugiere que los desarrolladores pueden enfocarse en estos géneros para atraer a esta audiencia.

- **Europa (UE)**: Los géneros de **Acción**, **Deportes** y **Carreras** lideran en esta región. A diferencia de NA, Europa presenta una inclinación ligeramente mayor hacia juegos de simulación y competencia, como los de carreras, lo cual sugiere una demanda por experiencias de simulación de vida y deportes.

- **Japón (JP)**: En Japón, los géneros de **Rol (RPG)** y **Acción** son los favoritos, con una menor presencia de géneros como Shooter. La preferencia por juegos de rol, especialmente aquellos creados por desarrolladores japoneses, refleja una fuerte cultura de storytelling y aventura en el mercado japonés.

**Diferencias Regionales en Preferencias de Género**:
Estas diferencias resaltan la necesidad de adaptar los lanzamientos y campañas de marketing para cada región. Los juegos de rol son especialmente atractivos en Japón, mientras que en NA y UE los juegos de acción y deportes tienen mayor éxito, lo que debe tomarse en cuenta para maximizar el éxito en cada mercado.

**Impacto de las Clasificaciones ESRB en Ventas por Región**

- **Norteamérica (NA)**: Las clasificaciones ESRB tienen un impacto significativo en las ventas, especialmente en las categorías **"E" (Everyone)** y **"M" (Mature)**. La categoría "M" es particularmente bien recibida en esta región, lo cual refleja la popularidad de los géneros de acción y shooter en un mercado que demanda experiencias maduras y de ritmo rápido.

- **Europa (UE)**: Las ventas por clasificación siguen una tendencia similar a NA, con ventas destacadas en los juegos clasificados como "E" y "M". Esto indica una amplia base de usuarios de diferentes edades y preferencias de contenido, lo que permite que la industria explore diferentes estilos y temáticas de juego.

- **Japón (JP)**: Las clasificaciones ESRB tienen un impacto más moderado, ya que los juegos orientados a todas las edades y jóvenes muestran mejor rendimiento en ventas. Esto es consistente con la popularidad de consolas portátiles y géneros familiares, que son más accesibles para un público amplio y casual en Japón.

**Diferencias Regionales en el Impacto de la Clasificación ESRB**:
En NA y UE, las clasificaciones ESRB influyen más en las ventas, mientras que en Japón su impacto es menor. Este hallazgo es relevante para el lanzamiento de juegos, ya que permite ajustar el contenido y clasificación de los productos según el perfil de audiencia regional.

**Resumen Intermedio**

1. **Preferencias de Plataforma**:
   - En NA y UE, Xbox y PlayStation son las plataformas líderes, mientras que en JP dominan las consolas de Nintendo. Estas diferencias sugieren que los esfuerzos de marketing deben enfocarse en las plataformas más exitosas de cada región para optimizar la efectividad de las campañas.

2. **Géneros Populares**:
   - Los géneros de **acción** y **deportes** son populares en NA y UE, mientras que en JP los géneros de **rol** y **acción** son preferidos. Esto permite a las empresas concentrar sus lanzamientos y esfuerzos de desarrollo en los géneros que resuenan mejor con cada audiencia regional.

3. **Impacto de Clasificación ESRB**:
   - La clasificación ESRB es más relevante en NA y UE que en JP, lo que indica que los consumidores en estas regiones pueden estar más influenciados por las categorías de contenido al momento de decidir sus compras.

**Recomendaciones Basadas en los Hallazgos**

- **Estrategias de Plataforma Regionalizadas**: Enfocar los esfuerzos de marketing y lanzamiento en las plataformas líderes de cada región para asegurar una mayor receptividad y maximizar el impacto en cada mercado.

- **Foco en Géneros Rentables**: Para NA y UE, priorizar juegos de acción y deportes, mientras que para JP sería conveniente enfocarse en juegos de rol. Esto permitirá satisfacer mejor las demandas específicas de cada mercado regional.

- **Adaptación de Clasificaciones**: En NA y UE, optimizar las estrategias de contenido de acuerdo con las clasificaciones ESRB, especialmente en los segmentos "M" y "E", mientras que en JP, los juegos accesibles para todas las edades son mejor recibidos.

Estos hallazgos permiten diseñar una estrategia de lanzamiento efectiva que se alinee con las preferencias locales, maximizando el éxito comercial en cada región y optimizando la experiencia del usuario en función de sus intereses y cultura de juego.

## Pruebas de hipótesis

Vamos a formular y comprobar dos hipótesis, utilizando una prueba estadística para cada una. 

Evaluaremos las hipótesis mediante la prueba t de Student, ideal para comparar las medias de dos muestras.

### Primera Hipótesis

**Objetivo**:  
Determinar si existen diferencias significativas en las calificaciones promedio otorgadas por los usuarios a los juegos en las plataformas Xbox One y PC. Esta comparación permitirá identificar si la percepción de calidad de los juegos difiere según la plataforma, lo cual puede ser de interés para desarrolladores y editores al decidir en qué plataformas lanzar o promocionar sus juegos.

**Formulación de la Hipótesis**:  
- **Hipótesis nula (H₀)**: Las calificaciones promedio de los usuarios para Xbox One y PC son iguales.
- **Hipótesis alternativa (H₁)**: Las calificaciones promedio de los usuarios para Xbox One y PC son diferentes.

**Prueba Estadística**:  
Utilizaremos la **prueba t de Student** para comparar las medias de las dos muestras de calificaciones de usuarios (Xbox One y PC). Esta prueba es adecuada para determinar si las diferencias observadas en las medias son estadísticamente significativas.

**Método**:
1. Realizar la prueba de Levene para verificar la igualdad de varianzas entre las calificaciones de ambas plataformas.
2. Usar el resultado de la prueba de Levene para decidir si se asumen o no varianzas iguales en la prueba t.
3. Aplicar la prueba t de Student y evaluar el valor p resultante.
4. Con un nivel de significancia de 0.05:
   - Si el valor p es menor que 0.05, rechazamos la hipótesis nula y concluimos que las calificaciones promedio de los usuarios para Xbox One y PC son diferentes.
   - Si el valor p es mayor o igual a 0.05, no rechazamos la hipótesis nula y concluimos que las calificaciones promedio de los usuarios para ambas plataformas son iguales.


In [ ]:
# Nivel de significancia
alpha = 0.05

# Verificar si la columna 'user_score' existe
if 'user_score' in data.columns:
    
    # Primera hipótesis: Comparación de calificaciones promedio para Xbox One y PC
    xbox_scores = data[data['platform'] == 'XOne']['user_score'].dropna()
    pc_scores = data[data['platform'] == 'PC']['user_score'].dropna()

    # Prueba de Levene para verificar igualdad de varianzas
    levene_stat, levene_p_value = stats.levene(xbox_scores, pc_scores)
    print("Prueba de Levene: Varianza de calificaciones para Xbox One vs PC")
    print(f"Estadístico Levene: {levene_stat:.2f}, Valor p: {levene_p_value:.4f}")

    # Determinar el valor de equal_var en función del resultado de Levene
    equal_var = levene_p_value >= alpha

    # Prueba t de Student
    t_stat, p_value = stats.ttest_ind(xbox_scores, pc_scores, equal_var=equal_var)
    print("\nPrueba de hipótesis: Calificaciones promedio para Xbox One vs PC")
    print(f"Estadístico t: {t_stat:.2f}, Valor p: {p_value:.4f}")
    if p_value < alpha:
        print("Rechazamos la hipótesis nula: Las calificaciones promedio son diferentes.")
    else:
        print("No se rechaza la hipótesis nula: Las calificaciones promedio son iguales.")


### Segunda Hipótesis

**Objetivo**:  
Evaluar si existen diferencias significativas en las calificaciones promedio de los usuarios entre los géneros de Acción y Deportes. Esta comparación ayudará a identificar si los usuarios perciben la calidad de los juegos de manera diferente según el género, lo cual podría ser útil para desarrolladores y editores en el diseño y la promoción de juegos que se alineen mejor con las expectativas y preferencias de la audiencia en cada categoría.

**Formulación de la Hipótesis**:  
- **Hipótesis nula (H₀)**: Las calificaciones promedio de los usuarios para los géneros de Acción y Deportes son iguales.
- **Hipótesis alternativa (H₁)**: Las calificaciones promedio de los usuarios para los géneros de Acción y Deportes son diferentes.

**Prueba Estadística**:  
Utilizaremos la **prueba t de Student** para comparar las medias de las calificaciones de usuarios entre los géneros de Acción y Deportes. Esta prueba es apropiada para determinar si las diferencias observadas en las medias son estadísticamente significativas.

**Método**:
1. Realizar la prueba de Levene para verificar la igualdad de varianzas entre las calificaciones de los géneros de Acción y Deportes.
2. Basándonos en el resultado de la prueba de Levene, decidir si asumimos o no varianzas iguales en la prueba t.
3. Aplicar la prueba t de Student y evaluar el valor p obtenido.
4. Con un nivel de significancia de 0.05:
   - Si el valor p es menor que 0.05, rechazamos la hipótesis nula y concluimos que las calificaciones promedio de los usuarios para los géneros de Acción y Deportes son diferentes.
   - Si el valor p es mayor o igual a 0.05, no rechazamos la hipótesis nula y concluimos que las calificaciones promedio de los usuarios para ambos géneros son iguales.

In [ ]:
from scipy import stats

# Nivel de significancia
alpha = 0.05

# Verificar si la columna 'user_score' existe
if 'user_score' in data.columns:
    
    # Segunda hipótesis: Comparación de calificaciones promedio para Acción y Deportes
    action_scores = data[data['genre'] == 'Action']['user_score'].dropna()
    sports_scores = data[data['genre'] == 'Sports']['user_score'].dropna()

    # Prueba de Levene para verificar igualdad de varianzas
    levene_stat_genre, levene_p_value_genre = stats.levene(action_scores, sports_scores)
    print("Prueba de Levene: Varianza de calificaciones para Acción vs Deportes")
    print(f"Estadístico Levene: {levene_stat_genre:.2f}, Valor p: {levene_p_value_genre:.4f}")

    # Determinar el valor de equal_var en función del resultado de Levene
    equal_var_genre = levene_p_value_genre >= alpha

    # Prueba t de Student
    t_stat_genre, p_value_genre = stats.ttest_ind(action_scores, sports_scores, equal_var=equal_var_genre)
    print("\nPrueba de hipótesis: Calificaciones promedio para Acción vs Deportes")
    print(f"Estadístico t: {t_stat_genre:.2f}, Valor p: {p_value_genre:.4f}")
    if p_value_genre < alpha:
        print("Rechazamos la hipótesis nula: Las calificaciones promedio son diferentes.")
    else:
        print("No se rechaza la hipótesis nula: Las calificaciones promedio son iguales.")

else:
    print("La columna 'user_score' no está presente en el dataset.")


### Conclusiones Intermedias "Pruebas de Hipótesis"

**Primera Hipótesis: Comparación de Calificaciones Promedio entre Xbox One y PC**

1. **Prueba de Levene para Igualdad de Varianzas**:
   - Resultado: El valor p de la prueba de Levene es 0.2694, lo cual es mayor al nivel de significancia de 0.05.
   - Conclusión: No rechazamos la hipótesis de igualdad de varianzas, por lo que asumimos que ambas muestras tienen varianzas iguales para aplicar la prueba t.

2. **Prueba t de Student**:
   - Resultado: El valor p de la prueba t es 0.0000, lo cual es menor al nivel de significancia de 0.05.
   - Conclusión: Rechazamos la hipótesis nula, lo que indica que existen diferencias significativas en las calificaciones promedio de los usuarios para las plataformas Xbox One y PC. Esto sugiere que los usuarios perciben de manera diferente los juegos en estas plataformas, lo que puede ser relevante para el desarrollo y marketing de productos específicos para cada plataforma.

**Segunda Hipótesis: Comparación de Calificaciones Promedio entre los Géneros de Acción y Deportes**

1. **Prueba de Levene para Igualdad de Varianzas**:
   - Resultado: El valor p de la prueba de Levene es 0.0002, lo cual es menor al nivel de significancia de 0.05.
   - Conclusión: Rechazamos la hipótesis de igualdad de varianzas, indicando que las muestras de calificaciones para los géneros de Acción y Deportes tienen varianzas diferentes. Este resultado guía la elección de la configuración `equal_var=False` en la prueba t de Student.

2. **Prueba t de Student**:
   - Resultado: El valor p de la prueba t es 0.1148, mayor al nivel de significancia de 0.05.
   - Conclusión: No rechazamos la hipótesis nula, lo que sugiere que no hay diferencias significativas en las calificaciones promedio de los usuarios para los géneros de Acción y Deportes. Esto indica que, en general, los usuarios valoran de manera similar los juegos de ambos géneros, lo cual podría ser útil para desarrolladores al decidir en qué género enfocar sus esfuerzos sin preocuparse por una gran diferencia en la percepción de calidad promedio.
   
Estas conclusiones intermedias sugieren que las plataformas pueden influir en la percepción de los usuarios, mientras que el género del juego no genera una diferencia significativa en calificaciones promedio.

## Conclusión general

Resume los hallazgos clave y proporciona recomendaciones específicas.


1. **Preparación de Datos y Transformación Inicial**
   - Se completaron adecuadamente los valores faltantes, logrando un conjunto de datos estandarizado y limpio. Las columnas con valores ausentes críticos, como `year_of_release`, se completaron utilizando la moda dentro de cada grupo por juego, mientras que otras columnas, como `user_score` y `critic_score`, fueron completadas usando la mediana. Las columnas de ventas regionales se llenaron con ceros, asumiendo que la falta de datos representa la ausencia de ventas en dichas regiones.
   - La conversión de tipos de datos y la estandarización de nombres de columnas mejoraron la consistencia y legibilidad, facilitando el análisis estadístico y comparativo.

2. **Tendencias de Lanzamiento y Ventas Globales**
   - El análisis histórico de lanzamientos mostró un crecimiento sostenido de la industria hasta 2008, seguido de una leve disminución, reflejando una transición hacia un mercado maduro. Esto sugiere que las estrategias futuras de desarrollo deberían enfocarse en la calidad y en adaptar los contenidos a las expectativas cambiantes de los usuarios, en lugar de la cantidad de lanzamientos.
   - La tendencia de ventas globales suavizadas destacó plataformas con rendimientos constantes, permitiendo observar aquellas que mantienen un éxito sostenido en el tiempo. Esto aporta una perspectiva importante para planificar lanzamientos en plataformas con estabilidad en el mercado.

3. **Preferencias de Plataforma y Género por Región**
   - Norteamérica y Europa comparten una preferencia por plataformas como Xbox y PlayStation, mientras que en Japón dominan las plataformas de Nintendo, especialmente las consolas portátiles. Estas diferencias regionales sugieren que las estrategias de marketing y desarrollo deben adaptarse a las preferencias de cada región, optimizando la inversión en plataformas más populares localmente.
   - En términos de género, Norteamérica y Europa prefieren géneros de acción, deportes y shooters, mientras que en Japón destacan los géneros de rol (RPG) y acción. Estos hallazgos sugieren enfoques diferenciados en el diseño y promoción de juegos, centrándose en géneros específicos según la región para atraer a las audiencias locales.

4. **Impacto de las Reseñas en las Ventas**
   - La correlación moderada entre las reseñas de críticos y las ventas indica que, aunque las reseñas positivas contribuyen a mayores ventas, no son el único factor determinante. Otros elementos, como el marketing y la accesibilidad, también influyen en el éxito de los juegos, destacando la importancia de una estrategia integral que combine calidad, marketing y disponibilidad para maximizar las ventas.

5. **Análisis de Clasificación ESRB por Región**
   - En Norteamérica y Europa, las clasificaciones ESRB tienen un impacto significativo, especialmente en las categorías "Mature" y "Everyone". En Japón, los juegos clasificados para todas las edades son más populares, lo que destaca la importancia de adaptar el contenido y las estrategias de lanzamiento en función de las normas culturales y las preferencias de cada mercado.

6. **Pruebas de Hipótesis**
   - La prueba de hipótesis entre las calificaciones promedio de Xbox One y PC reveló una diferencia significativa, lo que sugiere que los usuarios perciben la calidad de los juegos de manera distinta en estas plataformas. Esto podría influir en las decisiones de desarrollo y marketing en cuanto a las características exclusivas de cada plataforma.
   - No se encontraron diferencias significativas en las calificaciones promedio entre los géneros de Acción y Deportes, indicando que la percepción de calidad por género es similar. Esto sugiere que los desarrolladores pueden distribuir esfuerzos entre ambos géneros sin preocuparse por grandes diferencias en las calificaciones promedio.

**Conclusión Final**
En conjunto, este análisis ofrece una visión detallada de las tendencias y preferencias del mercado de videojuegos, tanto a nivel global como regional. Las empresas pueden usar estos hallazgos para diseñar estrategias de desarrollo, marketing y lanzamiento más efectivas, maximizando el impacto en cada región y optimizando la alineación con las preferencias del usuario en términos de plataforma, género y clasificación de contenido.